In [1]:
import tensorflow 
import pandas as pd
import pathlib

In [31]:
data = pd.read_csv('Questions/training_question.csv')

In [3]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds
#newsize=(300,400,100)
from PIL import Image

In [32]:
data = data[data ['Question_Type'] == 'Condition_Recognition']
data = data[data ['Image_ID'].isin(os.listdir('Images/Train_Image'))]

In [33]:
data.drop(labels=['_key','Question','Question_Type'],axis=1,inplace = True)

In [34]:
data.loc[data["Ground_Truth"] == "flooded", "Ground_Truth"] = 1
data.loc[data["Ground_Truth"] == "non flooded", "Ground_Truth"] = 0
data.drop_duplicates(keep=False, inplace=True)

In [35]:
x = list(data['Image_ID'])
b = os.listdir('Images/Images')
imgs=[]

In [36]:
l=[]
for i in b:
    if i in x:
        l.append(i)
list.sort(l)
l.remove('7693.JPG')
l.remove('9003.JPG')
data = data[data.Image_ID!= '9003.JPG']
data = data[data.Image_ID!= '7693.JPG']

In [37]:
del x
del b

In [38]:
imgs=[]
for i in range(len(l)):
    path = 'Images/Train_Image/' + l[i]
    Load_image=tf.keras.preprocessing.image.load_img(path)
    itr = np.array(Load_image)
    imgs.append(itr)

In [39]:
train_imgs =imgs[:100]
test_imgs = imgs [100:]
y = list(data['Ground_Truth'])
y_train = y[:100]
y_test = y[100:]
del imgs

In [40]:
del l

In [41]:
x_train1 = []
y_train1 = []
for i in range(len(train_imgs)):
    if len(train_imgs[i])==3000:
        x_train1.append(train_imgs[i])
        y_train1.append(y[i])
x_train1 = np.array(x_train1)
y_train1 = np.array(y_train1)

In [42]:
#x_train1 = np.expand_dims(x_train1, axis=0)
#y_train1 = np.expand_dims(y_train1, axis=0)
del y
del train_imgs
del y_test 
del test_imgs

In [43]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications.resnet50 import decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import Model

In [44]:
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array

In [45]:
from tensorflow.keras.layers import Input,Dense, MaxPool2D, Conv2D, Flatten
from tensorflow.keras import Sequential, Input

In [58]:
model=Sequential()

In [59]:
#model.add(Input([100,4000,3000,1]))
model.add(Conv2D(16, 3, activation='relu', input_shape=(3000,4000,3)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(32,3, activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(64, 3, activation='relu'))
model.add(MaxPool2D(2,2))
model.add(Flatten(input_shape=(3000,4000,3)))

In [60]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 2998, 3998, 16)    448       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 1499, 1999, 16)   0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 1497, 1997, 32)    4640      
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 748, 998, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 746, 996, 64)      18496     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 373, 498, 64)    

In [61]:
#model.add(Dense(128,activation='tanh'))
model.add(Dense(1,activation='sigmoid'))

In [62]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [63]:
model.fit(x=x_train1, y=y_train1, epochs=2, batch_size = 4)

Epoch 1/2
23/23 [==============================] - 93s 4s/step - loss: 0.6877 - accuracy: 1.0000
Epoch 2/2
23/23 [==============================] - 93s 4s/step - loss: 0.6764 - accuracy: 1.0000


In [64]:
model.save('flood_detection.h5')

In [66]:
import tensorflow as tf
model = tf.keras.models.load_model('flood_detection.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflmodel = converter.convert()
file = open( 'flood_detection.tflite' , 'wb' ) 
file.write( tflmodel )



INFO:tensorflow:Assets written to: /var/folders/b7/z5vwn3h52m3g5vn1rt3rm2v80000gp/T/tmp_357k4vh/assets


INFO:tensorflow:Assets written to: /var/folders/b7/z5vwn3h52m3g5vn1rt3rm2v80000gp/T/tmp_357k4vh/assets


47651116